# TESTE FUNÇÕES

## Bibliotecas

In [3]:
import pandas as pd
from openpyxl import load_workbook
import os
from datetime import datetime, timedelta
import pdfplumber
import re

## Funções

In [38]:
gestao_monitoramento = 'gestao_monitoramento.xlsx'
# salva formatação
workbook = load_workbook(gestao_monitoramento)
df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes', index_col=None)
df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

### certidao_cliente(df_clientes, df_cert_config)
cria um dataframe clientes 1:1 certidões

In [33]:
# ATUALIZADA 

def certidoes_clientes_ativo(df_clientes, df_cert_config):
    # declara variável para armazenadas dados para relatório
    global relatorio_global

    try:

        df_clientes['CERTIDOES_CADASTRADAS'] = df_clientes['CERTIDOES_CADASTRADAS'].astype(str)
        
        df_clientes_expandido = df_clientes.assign(
            CERTIDAO_ID=df_clientes['CERTIDOES_CADASTRADAS'].str.split(', ')
        ).explode('CERTIDAO_ID')

        df_clientes_certidoes = pd.merge(df_clientes_expandido,
                                    df_cert_config,
                                    on='CERTIDAO_ID', 
                                    how='left')[['CERTIDAO_ID', 'CLIENTE_ID', 'ATIVO', 'PASTA_DESTINO', 'NOME_ARQUIVO']]

        df_clientes_certidoes = df_clientes_certidoes.rename(columns={'NOME_ARQUIVO': 'ARQUIVO'})

        df_clientes_certidoes.info()
        
        # filtrando apenas onde o 'ATIVO' é True
        df_clientes_certidoes = df_clientes_certidoes.loc[df_clientes_certidoes['ATIVO'] == True]
        
        print('df_cliente_certidao criado com sucesso')
        return df_clientes_certidoes
    
    except Exception as e:
        relatorio = {
            'cod': 'FATAL01',
            'erro': 'Erro ao gerar df_cliente_certidao',
            'mensagem': f'''Não foi possível listar cliente 1:1 certidão.\n
            Verifique a existencia e relação dos dados de clientes e certidões.\nErro: {e}''',
            'dados': None
        }
        relatorio_global.append(relatorio)
        return None
    
df_cliente_certidao = certidoes_clientes_ativo(df_clientes, df_cert_config)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CERTIDAO_ID    54 non-null     object
 1   CLIENTE_ID     54 non-null     object
 2   ATIVO          54 non-null     bool  
 3   PASTA_DESTINO  54 non-null     object
 4   ARQUIVO        54 non-null     object
dtypes: bool(1), object(4)
memory usage: 1.9+ KB
df_cliente_certidao criado com sucesso


In [ ]:
df_cliente_certidao.info()
df_cliente_certidao.head(100)

### verificar_existencia_arquivos(dataframe)
recebe um DF (df_cliente_certidao ou df_monitor) e retorna df com coluna bool 'EXISTE'

In [37]:
# ATUALIZADA
def verificar_existencia_arquivos(dataframe):
    # declara variavel para armazenar dados para relatório 
    global relatorio_global 
    relatorio = []

    try:
        resultados = []
        pastas_grupo = dataframe.groupby('PASTA_DESTINO')

        for pasta_destino, grupo in pastas_grupo:
            # verificar se o diretório existe
            if not os.path.isdir(pasta_destino):
                for _, row in grupo.iterrows():
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'NOME_REF': row['ARQUIVO'],
                        'ARQUIVO_ENCONTRADO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
                
                relatorio = {
                    'cod': 'FATAL02',
                    'erro': 'Diretório não encontrado',
                    'mensagem': f'Não foi possível encontrar o diretório: {pasta_destino}',
                    'dados': None
                }
                continue

            # lista todos os arquivos da pasta_destino
            arquivos_na_pasta = os.listdir(pasta_destino)

            for _, row in grupo.iterrows():
                nome_arquivo = row['ARQUIVO']
                arquivos_encontrados = []

                for arquivo in arquivos_na_pasta:
                    if nome_arquivo in arquivo and arquivo.endswith('.pdf'):
                        arquivo_pdf = os.path.join(arquivo)
                        arquivos_encontrados.append(arquivo_pdf)

                # Adicionar uma entrada para cada arquivo encontrado
                if arquivos_encontrados:
                    for arquivo_pdf in arquivos_encontrados:
                        resultados.append({
                            'CERTIDAO_ID': row['CERTIDAO_ID'],
                            'CLIENTE_ID': row['CLIENTE_ID'],
                            'EXISTE': True,
                            'NOME_REF': row['ARQUIVO'],
                            'ARQUIVO_ENCONTRADO': arquivo_pdf,
                            'PASTA_DESTINO': pasta_destino
                        })
                else:
                    # Se nenhum arquivo foi encontrado, adicionar como não encontrado
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'NOME_REF': row['ARQUIVO'],
                        'ARQUIVO_ENCONTRADO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
        #cria df com os resultados encontrados
        df_cliente_certidoe_resultado = pd.DataFrame(resultados)
        return df_cliente_certidoe_resultado
    except Exception as e:
        print(f'Erro ao verificar a existência dos arquivos {e}')

df_cliente_certidao_resultado = verificar_existencia_arquivos(df_cliente_certidao)

In [36]:
df_cliente_certidao_resultado.info()
df_cliente_certidao_resultado.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CERTIDAO_ID         47 non-null     object
 1   CLIENTE_ID          47 non-null     object
 2   EXISTE              47 non-null     bool  
 3   NOME_REF            47 non-null     object
 4   ARQUIVO_ENCONTRADO  41 non-null     object
 5   PASTA_DESTINO       47 non-null     object
dtypes: bool(1), object(5)
memory usage: 2.0+ KB


,CERTIDAO_ID,CLIENTE_ID,EXISTE,NOME_REF,ARQUIVO_ENCONTRADO,PASTA_DESTINO
0,CEF-CRFGTS,AUTOLTDA,True,CR FGTS,2024-10-04_CR FGTS.pdf,clientes/autodata/
1,TJDFT-CNFR,AUTOLTDA,True,CN Falencia e Recuperacao,2024-09-15_CN Falencia e Recuperacao.pdf,clientes/autodata/
2,CGU-CNC,AUTOLTDA,True,"CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM)","2024-09-09_CN Correcional (ePAD, CGU-PJ, CEIS,...",clientes/autodata/
3,FN-CNDTF,AUTOLTDA,True,CND Tributos Federais,2024-09-15_CND Tributos Federais.pdf,clientes/autodata/
4,GDF-CND,AUTOLTDA,True,CND GDF,2024-09-06_CND GDF.pdf,clientes/autodata/
5,GDF-DIF,AUTOLTDA,True,Comprovante DIF GDF,2024-09-04_Comprovante DIF GDF.pdf,clientes/autodata/
6,TST-CNDT,AUTOLTDA,True,CND Trabalhista,2024-09-28_CND Trabalhista.pdf,clientes/autodata/
7,CNJ-CNIADM,AUTOLTDA,True,CN Impropidade Administrativa e Inelegibilidade,2024-09-23_CN Impropidade Administrativa e Ine...,clientes/autodata/
8,TESTE-TESTE,AUTOLTDA,False,TesteInexistente,None,clientes/autodata/
9,CEF-CRFGTS,BIGBOX,True,CR FGTS,2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf,clientes/bigbox/


### arquivos_novos_dbmonitor(df_monitor, dataframe)

In [ ]:
# def arquivos_novos_dbmonitor(df_monitor, dataframe):
#     horario_atual = datetime.now().replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')

#     certidao_nao_encontrada = []
#     certidao_adicionada = []
    

#     # Itera sobre cada linha do DataFrame df_resultados
#     for _, row in dataframe.iterrows():
#         certidao_id = row['CERTIDAO_ID']
#         cliente_id = row['CLIENTE_ID']
        
#         # condição de comparação
#         if row['EXISTE']:
#             arquivo = row['ARQUIVO']
#             pasta_destino = row['PASTA_DESTINO']
            
#             # Verifica existencia de linha com CERTIDAO_ID, CLIENTE_ID e ARQUIVO iguais
#             condicao_existente = (
#                 (df_monitor['CERTIDAO_ID'] == certidao_id) &
#                 (df_monitor['CLIENTE_ID'] == cliente_id) &
#                 (df_monitor['ARQUIVO'] == arquivo)
#             )
            
#             if df_monitor[condicao_existente].empty:
#                 novo_dado = {
#                     'CERTIDAO_ID': certidao_id,
#                     'CLIENTE_ID': cliente_id,
#                     'PASTA_DESTINO': pasta_destino,
#                     'ARQUIVO': arquivo,
#                     'STATUS': 'adicionado',
#                     'DATA_INICIO': None,
#                     'DATA_VENCIMENTO': None,
#                     'NUMERO_CERTIDAO': None,
#                     'NADA_CONSTA': None,
#                     'CREATED_AT': None,
#                     'UPDATED_AT': horario_atual
#                 }
#                 certidao_adicionada.append(novo_dado)
#                 df_monitor = pd.concat([df_monitor, pd.DataFrame([novo_dado])], ignore_index=True)

#         # Condição para quando 'EXISTE' é False
#         else:
#             condicao_existente = (
#                 (df_monitor['CERTIDAO_ID'] == certidao_id) & 
#                 (df_monitor['CLIENTE_ID'] == cliente_id) & 
#                 (df_monitor['ARQUIVO'] == row['ARQUIVO'])
#                 )
#             if not df_monitor[condicao_existente].empty and row['STATUS'] in ['monitorando', 'VENCIDO']:
#                 df_monitor.loc[condicao_existente, 'STATUS'] = 'ERROR'
#                 # registrar_erro('arquivos_novos_dbmonitor', {
#                 #     'CERTIDAO_ID': certidao_id,
#                 #     'CLIENTE_ID': cliente_id,
#                 #     'ARQUIVO': row['ARQUIVO'] })

#             certidao_nao_encontrada.append({
#                 'CERTIDAO_ID': certidao_id,
#                 'CLIENTE_ID': cliente_id,
#                 'PASTA_DESTINO': row['PASTA_DESTINO'] if pd.notnull(row['PASTA_DESTINO']) else None,
#                 'TIME_STAMP': horario_atual
#             })

#     # Exibe o conteúdo atualizado de df_monitor e a lista de certidões não encontradas
#     print(f'Certidões não encontradas:{len(certidao_nao_encontrada)}')
#     print(certidao_nao_encontrada)
#     print('df_monitor atualizado')
#     print(f'Certidões adicionadas:{len(certidao_adicionada)}')

#     return df_monitor

In [41]:
# ATUALIZANDO 
def arquivos_novos_dbmonitor(df_monitor, dataframe):
    horario_atual = datetime.now().replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')

    certidao_nao_encontrada = []
    certidao_adicionada = []
    

    # Itera sobre cada linha do DataFrame recebido
    for _, row in dataframe.iterrows():
        certidao_id = row['CERTIDAO_ID']
        cliente_id = row['CLIENTE_ID']
        
        # condição de comparação
        if row['EXISTE']:
            arquivo = row['ARQUIVO_ENCONTRADO']
            pasta_destino = row['PASTA_DESTINO']
            
            # Verifica existencia de linha com CERTIDAO_ID, CLIENTE_ID e ARQUIVO iguais
            condicao_existente = (
                (df_monitor['CERTIDAO_ID'] == certidao_id) &
                (df_monitor['CLIENTE_ID'] == cliente_id) &
                (df_monitor['ARQUIVO'] == arquivo)
            )
            
            # se não existir registro == condicao_existente adiciona novo registro ao df_monitor
            if df_monitor[condicao_existente].empty:
                novo_dado = {
                    'CERTIDAO_ID': certidao_id,
                    'CLIENTE_ID': cliente_id,
                    'PASTA_DESTINO': pasta_destino,
                    'ARQUIVO': arquivo,
                    'STATUS': 'adicionado',
                    'DATA_INICIO': None,
                    'DATA_VENCIMENTO': None,
                    'NUMERO_CERTIDAO': None,
                    'NADA_CONSTA': None,
                    'CREATED_AT': None,
                    'UPDATED_AT': horario_atual
                }
                certidao_adicionada.append(novo_dado)
                df_monitor = pd.concat([df_monitor, pd.DataFrame([novo_dado])], ignore_index=True)

        # Condição para quando 'EXISTE' é False
        else:
            condicao_existente = (
                (df_monitor['CERTIDAO_ID'] == certidao_id) & 
                (df_monitor['CLIENTE_ID'] == cliente_id) & 
                (df_monitor['ARQUIVO'] == row['ARQUIVO_ENCONTRADO'])
                )
            

            if not df_monitor[condicao_existente].empty and row['STATUS'] in ['monitorando', 'vencido']:
                df_monitor.loc[condicao_existente, 'STATUS'] = 'ERROR'
                # registrar_erro('arquivos_novos_dbmonitor', {
                #     'CERTIDAO_ID': certidao_id,
                #     'CLIENTE_ID': cliente_id,
                #     'ARQUIVO': row['ARQUIVO'] })

            certidao_nao_encontrada.append({
                'CERTIDAO_ID': certidao_id,
                'CLIENTE_ID': cliente_id,
                'PASTA_DESTINO': row['PASTA_DESTINO'] if pd.notnull(row['PASTA_DESTINO']) else None,
                'TIME_STAMP': horario_atual
            })

    # Exibe o conteúdo atualizado de df_monitor e a lista de certidões não encontradas
    print(f'Certidões não encontradas:{len(certidao_nao_encontrada)}')
    print(certidao_nao_encontrada)
    print('df_monitor atualizado')
    print(f'Certidões adicionadas:{len(certidao_adicionada)}')

    return df_monitor

In [168]:
df_monitor_final = arquivos_novos_dbmonitor(df_monitor, df_cliente_certidao_resultado)

Certidões não encontradas:8
[{'CERTIDAO_ID': 'CEF-CRFGTS', 'CLIENTE_ID': 'AUTOLTDA', 'PASTA_DESTINO': 'clientes/autodata/', 'TIME_STAMP': datetime.datetime(2024, 11, 18, 1, 26, 26)}, {'CERTIDAO_ID': 'TJDFT-CNFR', 'CLIENTE_ID': 'AUTOLTDA', 'PASTA_DESTINO': 'clientes/autodata/', 'TIME_STAMP': datetime.datetime(2024, 11, 18, 1, 26, 26)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'AUTOLTDA', 'PASTA_DESTINO': 'clientes/autodata/', 'TIME_STAMP': datetime.datetime(2024, 11, 18, 1, 26, 26)}, {'CERTIDAO_ID': 'GDF-CND', 'CLIENTE_ID': 'BIGBOX', 'PASTA_DESTINO': 'clientes/bigbox/', 'TIME_STAMP': datetime.datetime(2024, 11, 18, 1, 26, 26)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BIGBOX', 'PASTA_DESTINO': 'clientes/bigbox/', 'TIME_STAMP': datetime.datetime(2024, 11, 18, 1, 26, 26)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BOTMEI', 'PASTA_DESTINO': 'clientes/botmei/', 'TIME_STAMP': datetime.datetime(2024, 11, 18, 1, 26, 26)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'DROSARIO', 'P

C:\Users\User\AppData\Local\Temp\ipykernel_11960\2968264422.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_monitor = pd.concat([df_monitor, pd.DataFrame([novo_dado])], ignore_index=True)


### atualizar_planilha()
salva dataframe na planilha db_monitor

In [74]:
def atualizar_planilha(gestao_monitoramento, workbook, df_monitor):
    # Carrega o arquivo Excel
    workbook = load_workbook(gestao_monitoramento)
    
    # Seleciona a aba 'db_monitor'
    sheet = workbook['db_monitor']

    # Localiza a tabela 'fDB_monitor'
    table = None
    for tbl in sheet._tables.values():
        if tbl.name == 'fDB_monitor':
            table = tbl
            break

    if table is None:
        print("Tabela 'fDB_monitor' não encontrada.")
        return

    # Limpa todas as linhas da tabela (exceto o cabeçalho)
    start_row = table.ref.split(':')[0][1:]
    end_row = table.ref.split(':')[1][1:]
    start_col = table.ref.split(':')[0][0]
    end_col = table.ref.split(':')[1][0]

    for row in sheet.iter_rows(min_row=int(start_row) + 1, max_row=int(end_row), min_col=ord(start_col) - 64, max_col=ord(end_col) - 64):
        for cell in row:
            cell.value = None

    # Preenche os dados do DataFrame a partir da linha da tabela
    for index, row in df_monitor.iterrows():
        for col_num, value in enumerate(row, start=1):  # start=1 para a primeira coluna (A)
            sheet.cell(row=int(start_row) + 1 + index, column=col_num, value=value)  # Inicia na linha da tabela, logo abaixo do cabeçalho

    # Atualiza o intervalo da tabela para incluir todas as novas linhas
    table.ref = f"{start_col}{start_row}:{end_col}{int(start_row) + len(df_monitor)}"

    # Salva as alterações no arquivo Excel
    workbook.save(gestao_monitoramento)
    print("Arquivo Excel atualizado com os dados do DataFrame!")

# Exemplo de uso
gestao_monitoramento = 'gestao_monitoramento.xlsx'
workbook = load_workbook(gestao_monitoramento)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

# atualizar_planilha(gestao_monitoramento, workbook, df_monitor_final)

In [170]:
atualizar_planilha(gestao_monitoramento, workbook, df_monitor_final)

Arquivo Excel atualizado com os dados do DataFrame!


### ERRO

In [171]:
def registrar_erro(funcao, dados):
    global log_erros 
    log_erros.append({ 
        'funcao': funcao, 
        'dados': dados, 
        'timestamp': datetime.now().replace(microsecond=0)})

## Funções extração

In [43]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

pdf_path = 'clientes/drosario/CN Impropidade Administrativa e Inelegibilidade.pdf'

df_monitor.info()
df_monitor.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      15 non-null     object        
 1   CLIENTE_ID       15 non-null     object        
 2   PASTA_DESTINO    15 non-null     object        
 3   ARQUIVO          15 non-null     object        
 4   STATUS           15 non-null     object        
 5   DATA_INICIO      15 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  15 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  15 non-null     object        
 8   NADA_CONSTA      15 non-null     bool          
 9   CREATED_AT       15 non-null     datetime64[ns]
 10  UPDATED_AT       15 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 1.3+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,BIGBOX,clientes/bigbox/,2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-11-08,2024-12-07,2024110818421197249619,True,2024-11-16 02:51:45,2024-11-18 01:26:26
1,TJDFT-CNFR,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e R...,monitorando,2024-11-14,2024-12-14,2024.CTD.F2H1.0WMJ.OFPU.E6S2.0YBG,True,2024-11-16 02:51:45,2024-11-18 01:26:26
2,CGU-CNC,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (eP...,monitorando,2024-11-14,2024-12-14,qEcHlcYKFT1lcJeXQYcj,True,2024-11-16 02:51:45,2024-11-18 01:26:26
3,FN-CNDTF,BIGBOX,clientes/bigbox/,2025-04-14_(BIGBOX_FN-CNDTF)_CND Tributos Fede...,monitorando,2024-10-16,2025-04-14,71DD.4186.596B.A1FB,True,2024-11-16 02:51:45,2024-11-18 01:26:26
4,GDF-DIF,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_GDF-DIF)_Comprovante DIF GD...,monitorando,2024-11-14,2024-12-14,-não existe-,True,2024-11-16 02:51:45,2024-11-18 01:26:26


### data_criacao_pdf(pdf_path)

In [44]:
def data_criacao_pdf(pdf_path):
    try:
        timestamp =  os.path.getctime(pdf_path)
        created_at = datetime.fromtimestamp(timestamp).replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')
        return created_at
    except Exception as e:
        print(f'Erro ao obter a data de criação do arquivo: {e}')
        return None
    
# data_criacao_pdf(pdf_path)

### extrair_data_inicio(texto_pdf)

In [47]:
def extrair_data_inicio(texto_pdf):
    try:
        padrao_regex = [
            r'Validade:(\d{2}/\d{2}/\d{4})',
            r'Emitida gratuitamente pela internet em:\s*(\d{2}/\d{2}/\d{4})',
            r'Gerado em:\s*(\d{2}/\d{2}/\d{4})',
            r'Certidão emitida às\s*\d{2}:\d{2}:\d{2}\s*do dia\s*(\d{2}/\d{2}/\d{4})',
            r'Emitida às\s*\d{2}:\d{2}:\d{2}\s*do dia\s*(\d{2}/\d{2}/\d{4})',
            r'Expedição:\s*(\d{2}/\d{2}/\d{4})\s*, às\s*\d{2}:\d{2}:\d{2}',
            r'Este documento foi emitido no dia\s*(\d{2}/\d{2}/\d{4})',
            r'Certidão emitida via internet em\s*(\d{2}/\d{2}/\d{4})\s*às\s*\d{2}:\d{2}:\d{2}'
        ]

        for padrao in padrao_regex:
            par = re.search(padrao, texto_pdf)
            if par:
                data_encontrada = par.group(1)
                data_objeto = datetime.strptime(data_encontrada, '%d/%m/%Y')
                # data_formatada = data_objeto.strftime('%Y-%m-%d')
                return data_objeto

        print('Data de início não encontrada.')
        return None

    except Exception as e:
        print(f'Erro ao extrair data de início do arquivo: {e}')
        return None
    

# with pdfplumber.open(pdf_path) as pdf:
#                     texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)
# # texto_pdf = None
# extrair_data_inicio(texto_pdf)

### extrair_numero_certidao(texto_pdf, row)

In [48]:
def extrair_numero_certidao(texto_pdf, row):
    try:
        padrao_regex = [
            r'Certificação Número:\s*([\w./]+)',  # Ex: "Certificação Número: 024102105285887458243"
            r'Selo digital de segurança:\s*([\w./]+)',  # Ex: "Selo digital de segurança: 2024.CTD.ZZQ3.G1NW.0B85.GEDK.UR8P"
            r'número de controle\s*\n*([\w./]+)',  # Ex: "número de controle\n672D.DF85.D1C3.5589"
            r'Código de controle da certidão:\s*([\w./]+)',  # Ex: "Código de controle da certidão: YLkevKTcTH7aFG1S8seQ"
            r'Certidão nº:\s*([\w./]+)',  # Ex: "Certidão nº: 76418628/2024"
            r'CERTIDÃO Nº:\s*([\w./]+)'
        ]
        if row['CERTIDAO_ID'] == 'GDF-DIF':
            return '-não existe-'
        else:
            for padrao in padrao_regex:
                par = re.search(padrao, texto_pdf)
                if par:
                    numero_certidao = par.group(1)
                    return numero_certidao


    except Exception as e:
        print(f'Erro ao extrair número da certidão: {e}')
        return None

### extrair_nada_consta(texto_pdf)

In [49]:
def extrair_nada_consta(texto_pdf):
    try:
        padrao_regex = [
            r'NÃO\s*CONSTA',                    # "NÃO CONSTA"
            r'NADA\s*CONSTA',                   # "NADA CONSTA"
            r'encontra-se em situação regular', # "encontra-se em situação regular"
            r'não\s*constam',                   # "não constam"
            r'NÃO\s*CONSTAM',                   # "NÃO CONSTAM"
            r'ATIVO\s*\d{2}/\d{2}/\d{4}',       # Ativo - verifica cadastro ativo
            r'tem\s+os\s+mesmos\s+efeitos\s+da\s+certidão\s+negativa\.', # positiva com efeito de negativa
            r'certidão\s+positiva\s+de\s+débitos\s+com\s+efeito\s+de\s+negativa',
            r'certidão\s+positiva\s+com\s+efeito\s+de\s+negativa'
        ]

        for padrao in padrao_regex:
            par = re.search(padrao, texto_pdf)
            if par:
                return True
        
        return False
    
    except Exception as e:
        print(f'Erro ao extrair Nada Consta da certidão: {e}')
        return False

### extrair_data_vencimento(texto_pdf, data_inicio, row)

In [50]:
def extrair_data_vencimento(texto_pdf, row):
    data_inicio = row['DATA_INICIO']
    try:
        padrao_regex = [
            r"Validade:\s*\d{2}/\d{2}/\d{4}\s*a\s*(\d{2}/\d{2}/\d{4})",  # Ex: "Validade:21/10/2024 a 19/11/2024" (pegar a segunda data)
            r"Validade:\s*(\d{2}/\d{2}/\d{4})\s*",  # Ex: "Validade: 03/05/2025 -"
            r"Válida até\s*(\d{2}/\d{2}/\d{4})",  # Ex: "Válida até 14/04/2025"
            r"Válida até\s*(\d{2}) de (\w+) de (\d{4})",  # Ex: "Válida até 02 de fevereiro de 2025"
            r", com validade até o dia\s*(\d{2}/\d{2}/\d{4})"  # Ex: ", com validade até o dia 04/12/2024"
        ]
        
        meses = {
            'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
            'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
            'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
        }

        for padrao in padrao_regex:
            if row['CERTIDAO_ID'] in ('GDF-DIF','TJDFT-CNFR','CNJ-CNIADM'):
                print('TENTANDDO NOVA DATA AQUI')
                # data_objeto = datetime.strptime(data_inicio, '%Y-%m-%d')
                data_vencimento = data_inicio + timedelta(days=30)
                print(f'data_vencimento: {data_vencimento}')
                return data_vencimento
            
            par = re.search(padrao, texto_pdf)
            if par:
                # data por extenso
                if len(par.groups()) == 3:
                    dia, mes_extenso, ano = par.groups()
                    mes = meses.get(mes_extenso.lower()) # pega no dicionario
                    data_str = (f'{ano}-{mes}-{dia}')
                    data = datetime.strptime(data_str, '%Y-%m-%d')
                    return data
                else:
                    data_encontrada = par.group(1)
                    data_objeto = datetime.strptime(data_encontrada, '%d/%m/%Y')
                    # data_formatada = data_objeto.strftime('%Y-%m-%d')
                    return data_objeto  # Retorne a data encontrada
    except Exception as e:
        print(f'Erro ao extrair data de vencimento da certidão: {e}')
        return False

### renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config)

In [ ]:
# def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config):
    
#     # Capturar a linha onde 'NOME_ARQUIVO' é igual a 'certidao_id'
#     linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

#     if not linha_no_cert_config.empty:
#             # nome referencia
#             nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
            
#             arquivos_na_pasta = os.listdir(pasta_destino)

#             for arquivo in arquivos_na_pasta:
#                 if nome_arquivo_monitor in arquivo:

#                     indice_nome = nome_arquivo_monitor.find(nome_ref)
#                     if indice_nome != -1:
#                         print(f'nome original: {nome_arquivo_monitor}')
#                         nome_recortado = nome_arquivo_monitor[indice_nome:]
#                         print("Resultado:", nome_recortado)
                        
#                         # novo nome = data de vencimento e nome recortado
#                         data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
#                         novo_nome = f"{data_vencimento_str}_{nome_recortado}"
#                         novo_path = os.path.join(pasta_destino, novo_nome)
                        
#                         # Caminho do arquivo antigo
#                         arquivo_path = os.path.join(pasta_destino, nome_arquivo_monitor)
                        
#                         # Verifica se o novo nome já existe na pasta, incrementa o contador se necessário
#                         contador = 1
#                         while os.path.exists(novo_path):
#                             # Se o arquivo já existe, divide o nome e extensão e adiciona o contador
#                             novo_nome = f'{data_vencimento_str}({contador})_{nome_recortado}'
#                             novo_path = os.path.join(pasta_destino, novo_nome)
#                             print(novo_path)
#                             contador += 1

#                         # Renomeia o arquivo antigo para o novo nome
#                         try:
#                             os.rename(arquivo_path, novo_path)
#                             print(f"Arquivo renomeado para: {novo_path}")
#                             return novo_nome
#                         except FileNotFoundError:
#                             print(f"Arquivo original '{arquivo_path}' não encontrado.")
#                         except Exception as e:
#                             print(f"Erro ao renomear o arquivo: {e}")

In [ ]:
# import os
# from datetime import datetime

# def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config, status, cliente_id):
#     """
#     Renomeia ou move arquivos PDF com base no status ('adicionado', 'monitorando', 'vencido').

#     Args:
#         data_vencimento (datetime): Data de vencimento.
#         certidao_id (str): Identificador da certidão.
#         pasta_destino (str): Pasta onde os arquivos estão localizados.
#         nome_arquivo_monitor (str): Nome base do arquivo monitorado.
#         df_cert_config (DataFrame): Configurações da certidão.
#         status (str): Pode ser 'adicionado', 'monitorando' ou 'vencido'.
#         cliente_id (str, optional): Identificador do cliente (necessário para 'adicionado').

#     Returns:
#         str: Nome do arquivo atualizado ou mensagem indicando nenhuma ação realizada.
#     """
#     # Filtrar a linha no DataFrame
#     linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

#     if linha_no_cert_config.empty:
#         print("Certidão não encontrada no DataFrame.")
#         return None

#     nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
#     arquivos_na_pasta = os.listdir(pasta_destino)

#     for arquivo in arquivos_na_pasta:
#         if nome_arquivo_monitor in arquivo:
#             arquivo_path = os.path.join(pasta_destino, arquivo)

#             # Status: Adicionado
#             if status == 'adicionado':
#                 indice_nome = nome_arquivo_monitor.find(nome_ref)
#                 if indice_nome != -1:
#                     nome_recortado = nome_arquivo_monitor[indice_nome:]
#                     data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
#                     novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}"
#                     novo_path = os.path.join(pasta_destino, novo_nome)

#                     # Evitar sobrescrever arquivos existentes
#                     contador = 1
#                     while os.path.exists(novo_path):
#                         novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}({contador})"
#                         novo_path = os.path.join(pasta_destino, novo_nome)
#                         contador += 1

#                     # Renomear o arquivo
#                     try:
#                         os.rename(arquivo_path, novo_path)
#                         print(f"Arquivo renomeado para: {novo_path}")
#                         return novo_nome
#                     except Exception as e:
#                         print(f"Erro ao renomear arquivo: {e}")
#                         return None

#             # Status: Monitorando
#             elif status == 'monitorando':
#                 if nome_arquivo_monitor == arquivo:
#                     print(f"O arquivo {arquivo} já está sendo monitorado. Nenhuma ação necessária.")
#                     return "Nenhuma ação necessária"

#             # Status: Vencido
#             elif status == 'vencido':
#                 data_hora_atual = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
#                 nome_vencido = f"{os.path.splitext(arquivo)[0]}_b-{data_hora_atual}.pdf"
#                 nova_pasta = os.path.join(pasta_destino, "certidoes_vencidas")

#                 # Criar a pasta de vencidos, se não existir
#                 if not os.path.exists(nova_pasta):
#                     os.makedirs(nova_pasta)

#                 novo_path = os.path.join(nova_pasta, nome_vencido)

#                 # Mover o arquivo para a pasta de vencidos
#                 try:
#                     os.rename(arquivo_path, novo_path)
#                     print(f"Arquivo vencido movido para: {novo_path}")
#                     return nome_vencido
#                 except Exception as e:
#                     print(f"Erro ao mover arquivo vencido: {e}")
#                     return None

#     print(f"Nenhum arquivo correspondente encontrado para {nome_arquivo_monitor}.")
#     return None


#### novo rename e processa_monitor

In [117]:
df_monitor.columns

Index(['CERTIDAO_ID', 'CLIENTE_ID', 'PASTA_DESTINO', 'ARQUIVO', 'STATUS',
       'DATA_INICIO', 'DATA_VENCIMENTO', 'NUMERO_CERTIDAO', 'NADA_CONSTA',
       'CREATED_AT', 'UPDATED_AT'],
      dtype='object')

In [51]:
def renomear_pdf_path(row):
    print('iniciando renomear_pdf_patch()')
    # Carrega df_cert_config e filtra linha
    df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
    linha_certidao_id = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == row['CERTIDAO_ID']]

    if linha_certidao_id.empty:
        print(f"Certidão {row['CERTIDAO_ID']} não encontrada no config_certidoes.")
        return None
    
    nome_ref = linha_certidao_id['NOME_ARQUIVO'].values[0]
    arquivos_na_pasta = os.listdir(row['PASTA_DESTINO'])

    print(f'nome arquivo_monitor: {row["ARQUIVO"]}')
    for arquivo in arquivos_na_pasta:
        if row['ARQUIVO'] == arquivo and arquivo.endswith('.pdf'):
            arquivo_path = os.path.join(row['PASTA_DESTINO'], row['ARQUIVO'])
            print(f'nome arquivo na pasta: {arquivo}\n parquivo_path {arquivo_path}')
            indice_nome = row['ARQUIVO'].find(nome_ref)

            if indice_nome != -1: # .find retorna -1 se ñ encontrar
                    data_atual = datetime.now()
                    
                    if row['STATUS'] == 'adicionado':
                        
                        nome_recortado = row['ARQUIVO'][indice_nome:]
                        print(f'nome recortado {nome_recortado}')
                        data_vencimento_str = row['DATA_VENCIMENTO'].strftime('%Y-%m-%d')
                        novo_nome = f"{data_vencimento_str}_({row['CLIENTE_ID']}_{row['CERTIDAO_ID']})_{nome_recortado}"
                        path = row['PASTA_DESTINO']
                        novo_path = os.path.join(row["PASTA_DESTINO"], novo_nome)
                        row['STATUS'] = 'monitorando'
                        row['UPDATED_AT'] = data_atual
                        print(novo_nome)

                        if novo_nome != row['ARQUIVO']:
                            os.rename(arquivo_path, novo_path)
                            row['ARQUIVO'] = novo_nome
                            row['PASTA_DESTINO'] = path
                            row['STATUS'] = 'monitorando'

                            print('O nome é diferente!')
                        
                        return row

                    ## se VENCIDO    
                    if row['STATUS'] == 'vencido':
                        if 'certidoes_arquivadas' in row['PASTA_DESTINO']:
                            print('CERTIDÃO JÁ FOI ARQUIVADA')
                            return row
                        data_atual_str = data_atual.strftime('%Y-%m-%d_%H-%M')
                        nome, extensao = os.path.splitext(arquivo)
                        novo_nome = f'{nome}_a{data_atual_str}{extensao}'
                        nova_pasta = os.path.join(row['PASTA_DESTINO'], 'certidoes_arquivadas/')

                         # Criar a pasta de vencidos, se não existir
                        if not os.path.exists(nova_pasta):
                            os.makedirs(nova_pasta)
                    
                        novo_path = os.path.join(nova_pasta, novo_nome)

                        # Mover o arquivo para a pasta de vencidos
                        try:
                            os.rename(arquivo_path, novo_path)
                            print(f"Arquivo vencido movido para: {novo_path}")
                            row['PASTA_DESTINO'] = nova_pasta
                            row['ARQUIVO'] = novo_nome
                            row['UPDATED_AT'] = data_atual
                            return row
                        
                        except Exception as e:
                            print(f"Erro ao mover arquivo vencido: {e}")
                            return row


                         
            
    print()

    




In [374]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')
df_monitor = df_monitor.dropna(how='all').reset_index(drop=True)
df_monitor.info()
df_monitor.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      15 non-null     object        
 1   CLIENTE_ID       15 non-null     object        
 2   PASTA_DESTINO    15 non-null     object        
 3   ARQUIVO          15 non-null     object        
 4   STATUS           15 non-null     object        
 5   DATA_INICIO      15 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  15 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  15 non-null     object        
 8   NADA_CONSTA      15 non-null     bool          
 9   CREATED_AT       15 non-null     datetime64[ns]
 10  UPDATED_AT       15 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 1.3+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,BIGBOX,clientes/bigbox/,2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-11-08,2024-12-07,2024110818421197249619,True,2024-11-16 02:51:45,2024-11-18 01:26:26
1,TJDFT-CNFR,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e R...,monitorando,2024-11-14,2024-12-14,2024.CTD.F2H1.0WMJ.OFPU.E6S2.0YBG,True,2024-11-16 02:51:45,2024-11-18 01:26:26
2,CGU-CNC,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (eP...,monitorando,2024-11-14,2024-12-14,qEcHlcYKFT1lcJeXQYcj,True,2024-11-16 02:51:45,2024-11-18 01:26:26
3,FN-CNDTF,BIGBOX,clientes/bigbox/,2025-04-14_(BIGBOX_FN-CNDTF)_CND Tributos Fede...,monitorando,2024-10-16,2025-04-14,71DD.4186.596B.A1FB,True,2024-11-16 02:51:45,2024-11-18 01:26:26
4,GDF-DIF,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_GDF-DIF)_Comprovante DIF GD...,monitorando,2024-11-14,2024-12-14,-não existe-,True,2024-11-16 02:51:45,2024-11-18 01:26:26


In [225]:
gestao_monitoramento = 'gestao_monitoramento.xlsx'
# salva formatação
workbook = load_workbook(gestao_monitoramento)
df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes', index_col=None)
df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
# df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

## PROCESSA ADICIONADOS
def processa_adicionados(row):

    pdf_path = os.path.join(row['PASTA_DESTINO'], row['ARQUIVO'])

    with pdfplumber.open(pdf_path) as pdf:
        texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)

        row['CREATED_AT'] = data_criacao_pdf(pdf_path)

        row['DATA_INICIO'] = extrair_data_inicio(texto_pdf)

        row['NUMERO_CERTIDAO'] = extrair_numero_certidao(texto_pdf, row)

        row['NADA_CONSTA'] = extrair_nada_consta(texto_pdf)

        row['DATA_VENCIMENTO'] = extrair_data_vencimento(texto_pdf, row)

    row = renomear_pdf_path(row)
        
    print()

    return row


for index, row in df_monitor.iterrows():
    if row['STATUS'] == 'adicionado':
        # processa e atualiza a row
        row = processa_adicionados(row)
        df_monitor.iloc[index] = row

    if row['STATUS'] == 'vencido':
        print('Olá')
        print(row['PASTA_DESTINO'])
        print(row['ARQUIVO'])
        df_monitor.iloc[index] = renomear_pdf_path(row)
        print('novo')
        print(row['PASTA_DESTINO'])
        print(row['ARQUIVO'])
    # elif row['STATUS'] == 'VENCIDO':
    #     print('Tchau')
    # print(row)

Olá
clientes/botmei/certidoes_arquivadas/
2024-10-02_(BOTMEI_TJDFT-CNFR)_CN Falencia e Recuperacao_a2024-11-18_01-43.pdf
iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-10-02_(BOTMEI_TJDFT-CNFR)_CN Falencia e Recuperacao_a2024-11-18_01-43.pdf
nome arquivo na pasta: 2024-10-02_(BOTMEI_TJDFT-CNFR)_CN Falencia e Recuperacao_a2024-11-18_01-43.pdf
 parquivo_path clientes/botmei/certidoes_arquivadas/2024-10-02_(BOTMEI_TJDFT-CNFR)_CN Falencia e Recuperacao_a2024-11-18_01-43.pdf
CERTIDÃO JÁ FOI ARQUIVADA
novo
clientes/botmei/certidoes_arquivadas/
2024-10-02_(BOTMEI_TJDFT-CNFR)_CN Falencia e Recuperacao_a2024-11-18_01-43.pdf
Olá
clientes/botmei/certidoes_arquivadas/
2024-10-16_(BOTMEI_GDF-DIF)_Comprovante DIF GDF_a2024-11-18_01-43.pdf
iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-10-16_(BOTMEI_GDF-DIF)_Comprovante DIF GDF_a2024-11-18_01-43.pdf
nome arquivo na pasta: 2024-10-16_(BOTMEI_GDF-DIF)_Comprovante DIF GDF_a2024-11-18_01-43.pdf
 parquivo_path clientes/botmei/certido

In [228]:
df_monitor = df_monitor.dropna(how='all').reset_index(drop=True)
df_monitor['NADA_CONSTA'] = df_monitor['NADA_CONSTA'].astype(bool)
df_monitor[['DATA_INICIO', 'DATA_VENCIMENTO', 
            'CREATED_AT', 'UPDATED_AT']] = df_monitor[['DATA_INICIO', 'DATA_VENCIMENTO', 
                                                       'CREATED_AT', 'UPDATED_AT']].apply(pd.to_datetime)
df_monitor.info()
df_monitor.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      17 non-null     object        
 1   CLIENTE_ID       17 non-null     object        
 2   PASTA_DESTINO    17 non-null     object        
 3   ARQUIVO          17 non-null     object        
 4   STATUS           17 non-null     object        
 5   DATA_INICIO      17 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  17 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  17 non-null     object        
 8   NADA_CONSTA      17 non-null     bool          
 9   CREATED_AT       17 non-null     datetime64[ns]
 10  UPDATED_AT       17 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 1.5+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,BIGBOX,clientes/bigbox/,2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-11-08,2024-12-07,2024110818421197249619,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
1,TJDFT-CNFR,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e R...,monitorando,2024-11-14,2024-12-14,2024.CTD.F2H1.0WMJ.OFPU.E6S2.0YBG,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
2,CGU-CNC,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (eP...,monitorando,2024-11-14,2024-12-14,qEcHlcYKFT1lcJeXQYcj,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
3,FN-CNDTF,BIGBOX,clientes/bigbox/,2025-04-14_(BIGBOX_FN-CNDTF)_CND Tributos Fede...,monitorando,2024-10-16,2025-04-14,71DD.4186.596B.A1FB,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
4,GDF-DIF,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_GDF-DIF)_Comprovante DIF GD...,monitorando,2024-11-14,2024-12-14,-não existe-,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
5,TST-CNDT,BIGBOX,clientes/bigbox/,2025-05-13_(BIGBOX_TST-CNDT)_CND Trabalhista.pdf,monitorando,2024-11-14,2025-05-13,78855115/2024,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
6,CNJ-CNIADM,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CNJ-CNIADM)_CN Impropidade ...,monitorando,2024-11-14,2024-12-14,6735.7332.D1E2.8146,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000
7,TJDFT-CNFR,BOTMEI,clientes/botmei/certidoes_arquivadas/,2024-10-02_(BOTMEI_TJDFT-CNFR)_CN Falencia e R...,vencido,2024-09-02,2024-10-02,DLUML9MX30,True,2024-11-16 02:51:45,2024-11-18 01:57:21.805
8,GDF-DIF,BOTMEI,clientes/botmei/certidoes_arquivadas/,2024-10-16_(BOTMEI_GDF-DIF)_Comprovante DIF GD...,vencido,2024-09-16,2024-10-16,-não existe-,False,2024-11-16 02:51:45,2024-11-18 01:57:21.805
9,CEF-CRFGTS,DROSARIO,clientes/drosario/,2024-11-29_(DROSARIO_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-10-31,2024-11-29,2024103110152970642061,True,2024-11-16 02:51:45,2024-11-18 01:26:26.000


In [63]:
for index, row in df_monitor.iterrows():
    print('oia')
    print(row['PASTA_DESTINO'])
    print(row['ARQUIVO'])

oia
clientes/autodata/
2024-11-10_(AUTOLTDA_CEF-CRFGTS)_CR FGTS.pdf
oia
clientes/autodata/
CN Falencia e Recuperacao.pdf
oia
clientes/autodata/
2024-11-12_(AUTOLTDA_CGU-CNC)_CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM).pdf
oia
clientes/autodata/
2024-11-13_(AUTOLTDA_FN-CNDTF)_CND Tributos Federais.pdf
oia
clientes/autodata/
2024-11-17_(AUTOLTDA_GDF-CND)_CND GDF.pdf
oia
clientes/autodata/
Comprovante DIF GDF.pdf
oia
clientes/autodata/
2024-11-14_(AUTOLTDA_TST-CNDT)_CND Trabalhista.pdf
oia
clientes/autodata/
2024-10-25_(AUTOLTDA_CNJ-CNIADM)_CN Impropidade Administrativa e Inelegibilidade.pdf
oia
clientes/bigbox/
2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf
oia
clientes/bigbox/
2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e Recuperacao.pdf
oia
clientes/bigbox/
2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM).pdf
oia
clientes/bigbox/
2025-04-14_(BIGBOX_FN-CNDTF)_CND Tributos Federais.pdf
oia
clientes/bigbox/
2024-12-14_(BIGBOX_GDF-DIF)_Comprovante DIF GDF.pdf


In [229]:
atualizar_planilha(gestao_monitoramento, workbook, df_monitor)

Arquivo Excel atualizado com os dados do DataFrame!


### processa_monitor(df_monitor)

In [ ]:
# def processa_monitor(df_monitor):
#     horario_atual = datetime.now()

#     # Itera no DF 
#     for index, row in df_monitor.iterrows():
#         if row['STATUS'] == 'adicionado':
#             pdf_path = os.path.join(row['PASTA_DESTINO'], row['ARQUIVO'])
#             print(pdf_path)

#             # Inicializa variáveis para cada coluna que pode ser preenchida
#             cliente_id = ['CLIENTE_ID']
#             status = row['STATUS']
#             pasta_destino = row['PASTA_DESTINO']
#             certidao_id = row['CERTIDAO_ID']
#             nome_arquivo_monitor = row['ARQUIVO']
#             created_at = row['CREATED_AT']
#             data_inicio = row['DATA_INICIO']
#             numero_certidao = row['NUMERO_CERTIDAO']
#             nada_consta = row['NADA_CONSTA']
#             data_vencimento = row['DATA_VENCIMENTO']

#             # Flag de erro para controlar a execução geral
#             error_flag = 0  # Começa com 0, indicando que tudo está bem

#             try:
#                 # Abre o PDF e extrai o texto
#                 with pdfplumber.open(pdf_path) as pdf:
#                     texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)

#                 # Verifica e preenche cada coluna individualmente se estiver vazia

#                 # DATA DE CRIAÇÃO
#                 if pd.isna(row['CREATED_AT']) or row['CREATED_AT'] == 'nan':
#                     try:
#                         created_at = data_criacao_pdf(pdf_path)
#                         print(f'created_at: {created_at}')
#                         df_monitor.at[index, 'CREATED_AT'] = created_at

#                     except Exception as e:
#                         print(f"Erro ao extrair created_at: {e}")

#                 # DATA DE INÍCIO
#                 if pd.isna(row['DATA_INICIO']) or row['DATA_INICIO'] == 'nan':
#                     try:
#                         data_inicio = extrair_data_inicio(texto_pdf)
#                         print(f'data_inicio: {data_inicio}')
#                         df_monitor.at[index, 'DATA_INICIO'] = data_inicio

#                     except Exception as e:
#                         print(f"Erro ao extrair data_inicio: {e}")
                
#                 # NÚMERO DA CERTIDÃO
#                 if pd.isna(row['NUMERO_CERTIDAO']) or row['NUMERO_CERTIDAO'] == 'nan':
#                     try:
#                         numero_certidao = extrair_numero_certidao(texto_pdf, row)
#                         print(f'numero_certidao: {numero_certidao}')
#                     except Exception as e:
#                         print(f"Erro ao extrair numero_certidao: {e}")

#                 # NADA CONSTA
#                 if pd.isna(row['NADA_CONSTA']) or row['NADA_CONSTA'] == 'nan':
#                     try:
#                         nada_consta = extrair_nada_consta(texto_pdf)
#                         print(f'nada_consta: {nada_consta}')
#                     except Exception as e:
#                         print(f"Erro ao extrair nada_consta: {e}")

#                 # DATA DE VENCIMENTO
#                 if pd.isna(row['DATA_VENCIMENTO']) or row['DATA_VENCIMENTO'] == 'nan':
#                     try:
#                         data_vencimento = extrair_data_vencimento(texto_pdf, data_inicio, row)
#                         print(f'data_vencimento: {data_vencimento}')
#                     except Exception as e:
#                         print(f"Erro ao extrair data_vencimento: {e}")

#                 # RENOMEANDO ARQUIVO
#                 novo_nome = renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config, status, cliente_id)



#             except Exception as e:
#                 print(f"Erro ao processar o arquivo {pdf_path}: {e}")

#             finally:
#                 df_monitor.at[index, 'STATUS'] = 'monitorando'
#                 df_monitor.at[index, 'DATA_VENCIMENTO'] = data_vencimento
#                 df_monitor.at[index, 'NUMERO_CERTIDAO'] = numero_certidao
#                 df_monitor.at[index, 'NADA_CONSTA'] = nada_consta
#                 df_monitor.at[index, 'UPDATED_AT'] = horario_atual
#                 if novo_nome == None:
#                     continue
#                 else:
#                     df_monitor.at[index, 'ARQUIVO'] = novo_nome
                
#                 # if error_flag == 1:
#                 #     print('error flag 1')
#                 # else:
#                 #     print(f"Atualização parcial realizada para o index {index}, devido ao erro_flag.")

#     return df_monitor
    

# df_monitor = processa_monitor(df_monitor)

# df_monitor.info()

In [ ]:
df_monitor

In [ ]:
# import os
# from datetime import datetime

# def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config, status, cliente_id=None):

#     # Filtrar a linha no DataFrame
#     linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

#     if linha_no_cert_config.empty:
#         print("Certidão não encontrada no DataFrame.")
#         return None

#     nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
#     arquivos_na_pasta = os.listdir(pasta_destino)

#     for arquivo in arquivos_na_pasta:
#         if nome_arquivo_monitor in arquivo:
#             arquivo_path = os.path.join(pasta_destino, arquivo)

            
#             if status == 'adicionado':
#                 indice_nome = nome_arquivo_monitor.find(nome_ref)
#                 if indice_nome != -1:
#                     nome_recortado = nome_arquivo_monitor[indice_nome:]
#                     data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
#                     novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}"
#                     novo_path = os.path.join(pasta_destino, novo_nome)

#                     # Evitar sobrescrever arquivos existentes
#                     contador = 1
#                     while os.path.exists(novo_path):
#                         novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}({contador})"
#                         novo_path = os.path.join(pasta_destino, novo_nome)
#                         contador += 1

#                     # Renomear o arquivo
#                     try:
#                         os.rename(arquivo_path, novo_path)
#                         print(f"Arquivo renomeado para: {novo_path}")
#                         return novo_nome
#                     except Exception as e:
#                         print(f"Erro ao renomear arquivo: {e}")
#                         return None

#             # Status: Monitorando
#             elif status == 'monitorando':
#                 if nome_arquivo_monitor == arquivo:
#                     print(f"O arquivo {arquivo} já está sendo monitorado. Nenhuma ação necessária.")
#                     return "Nenhuma ação necessária"

#             # Status: Vencido
#             elif status == 'vencido':
#                 data_hora_atual = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
#                 nome_vencido = f"{os.path.splitext(arquivo)[0]}_b-{data_hora_atual}.pdf"
#                 nova_pasta = os.path.join(pasta_destino, "certidoes_vencidas")

#                 # Criar a pasta de vencidos, se não existir
#                 if not os.path.exists(nova_pasta):
#                     os.makedirs(nova_pasta)

#                 novo_path = os.path.join(nova_pasta, nome_vencido)

#                 # Mover o arquivo para a pasta de vencidos
#                 try:
#                     os.rename(arquivo_path, novo_path)
#                     print(f"Arquivo vencido movido para: {novo_path}")
#                     return nome_vencido
#                 except Exception as e:
#                     print(f"Erro ao mover arquivo vencido: {e}")
#                     return None

#     print(f"Nenhum arquivo correspondente encontrado para {nome_arquivo_monitor}.")
#     return None


In [220]:
df_monitor['NADA_CONSTA'] = df_monitor['NADA_CONSTA'].astype(bool)
df_monitor.info()
df_monitor.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      39 non-null     object        
 1   CLIENTE_ID       39 non-null     object        
 2   PASTA_DESTINO    39 non-null     object        
 3   ARQUIVO          39 non-null     object        
 4   STATUS           39 non-null     object        
 5   DATA_INICIO      39 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  39 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  39 non-null     object        
 8   NADA_CONSTA      39 non-null     bool          
 9   CREATED_AT       39 non-null     datetime64[ns]
 10  UPDATED_AT       39 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 3.2+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-10_CR FGTS.pdf,VENCIDA,2024-09-30,2024-11-10,SZFHLST1NP,True,2024-11-16 02:51:45,2024-11-17 01:26:31.440307
1,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-10-09_CN Falencia e Recuperacao.pdf,VENCIDA,2024-09-09,2024-10-09,85J6FHJEPA,True,2024-11-16 02:51:45,2024-11-17 01:26:31.462602
2,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-11-12_CN Correcional (ePAD, CGU-PJ, CEIS,...",VENCIDA,2024-09-13,2024-11-12,XJTS5TBKHE,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
3,FN-CNDTF,AUTOLTDA,clientes/autodata/,2024-11-13_CND Tributos Federais.pdf,VENCIDA,2024-09-06,2024-11-13,JLKPCGL1UU,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
4,GDF-CND,AUTOLTDA,clientes/autodata/,2024-11-17_CND GDF.pdf,VENCIDA,2024-09-20,2024-11-17,LKTPKFYZNQ,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605


In [238]:
atualizar_planilha(gestao_monitoramento, workbook, df_monitor)

Arquivo Excel atualizado com os dados do DataFrame!


### vencida

In [233]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')
df_monitor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      17 non-null     object        
 1   CLIENTE_ID       17 non-null     object        
 2   PASTA_DESTINO    17 non-null     object        
 3   ARQUIVO          17 non-null     object        
 4   STATUS           17 non-null     object        
 5   DATA_INICIO      17 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  17 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  17 non-null     object        
 8   NADA_CONSTA      17 non-null     bool          
 9   CREATED_AT       17 non-null     datetime64[ns]
 10  UPDATED_AT       17 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 1.5+ KB


In [235]:
indices_para_remover = []
for index, row in df_monitor.iterrows():
    data_atual = datetime.now()
    if row['STATUS'] == 'monitorando' and row['DATA_VENCIMENTO'] <= data_atual:
        df_monitor.at[index, 'STATUS'] = 'vencido'
        df_monitor.at[index, 'UPDATED_AT'] = data_atual
    if 'certidoes_arquivadas' in row['PASTA_DESTINO']:
        indices_para_remover.append(index)

print(indices_para_remover)


[7, 8]


In [236]:
df_monitor = df_monitor.drop(indices_para_remover).reset_index(drop=True)
df_monitor

,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,BIGBOX,clientes/bigbox/,2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-11-08,2024-12-07,2024110818421197249619,True,2024-11-16 02:51:45,2024-11-18 01:26:26
1,TJDFT-CNFR,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e R...,monitorando,2024-11-14,2024-12-14,2024.CTD.F2H1.0WMJ.OFPU.E6S2.0YBG,True,2024-11-16 02:51:45,2024-11-18 01:26:26
2,CGU-CNC,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (eP...,monitorando,2024-11-14,2024-12-14,qEcHlcYKFT1lcJeXQYcj,True,2024-11-16 02:51:45,2024-11-18 01:26:26
3,FN-CNDTF,BIGBOX,clientes/bigbox/,2025-04-14_(BIGBOX_FN-CNDTF)_CND Tributos Fede...,monitorando,2024-10-16,2025-04-14,71DD.4186.596B.A1FB,True,2024-11-16 02:51:45,2024-11-18 01:26:26
4,GDF-DIF,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_GDF-DIF)_Comprovante DIF GD...,monitorando,2024-11-14,2024-12-14,-não existe-,True,2024-11-16 02:51:45,2024-11-18 01:26:26
5,TST-CNDT,BIGBOX,clientes/bigbox/,2025-05-13_(BIGBOX_TST-CNDT)_CND Trabalhista.pdf,monitorando,2024-11-14,2025-05-13,78855115/2024,True,2024-11-16 02:51:45,2024-11-18 01:26:26
6,CNJ-CNIADM,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CNJ-CNIADM)_CN Impropidade ...,monitorando,2024-11-14,2024-12-14,6735.7332.D1E2.8146,True,2024-11-16 02:51:45,2024-11-18 01:26:26
7,CEF-CRFGTS,DROSARIO,clientes/drosario/,2024-11-29_(DROSARIO_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-10-31,2024-11-29,2024103110152970642061,True,2024-11-16 02:51:45,2024-11-18 01:26:26
8,TJDFT-CNFR,DROSARIO,clientes/drosario/,2024-12-14_(DROSARIO_TJDFT-CNFR)_CN Falencia e...,monitorando,2024-11-14,2024-12-14,2024.CTD.ZTPK.TE86.3MSB.AT0F.SMCG,True,2024-11-16 02:51:45,2024-11-18 01:26:26
9,CGU-CNC,DROSARIO,clientes/drosario/,2024-12-14_(DROSARIO_CGU-CNC)_CN Correcional (...,monitorando,2024-11-14,2024-12-14,Frp6kCRkte8Vkevk9Lar,True,2024-11-16 02:51:45,2024-11-18 01:26:26


# Teste principais


In [26]:
from winotify import Notification

toast = Notification(app_id="windows app",
                     title="Winotify Test Toast",
                     msg="New Notification!",)

toast.show()


In [68]:
from plyer import notification

def enviar_notificacao(titulo, mensagem, duracao=10):
    notification.notify(
        title=titulo,
        message=mensagem,
        timeout=duracao,
        app_name='Monitor de Certidões',
        app_icon='sign-error-icon_34362.ico'
    )

# Exemplo de uso
enviar_notificacao("Permission Denied", "Fechar o arquivo.", 10)


In [23]:
relatorio_global = []


In [53]:
print(len(relatorio_global))
print(relatorio_global)

1
[{'cod': 'TESTE', 'erro': 'Certidão não Encontrada', 'mensagem': 'As seguintes certidões não foram encontradas nos arquivos', 'dados': [{'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'AUTOLTDA', 'EXISTE': False, 'NOME_REF': 'TesteInexistente', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/autodata/'}, {'CERTIDAO_ID': 'GDF-CND', 'CLIENTE_ID': 'BIGBOX', 'EXISTE': False, 'NOME_REF': 'CND GDF', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/bigbox/'}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BIGBOX', 'EXISTE': False, 'NOME_REF': 'TesteInexistente', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/bigbox/'}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BOTMEI', 'EXISTE': False, 'NOME_REF': 'TesteInexistente', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/botmei/'}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'DROSARIO', 'EXISTE': False, 'NOME_REF': 'TesteInexistente', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/drosario/'}, {'CERTIDAO_ID': '

In [77]:
gestao_monitoramento = ['config_bot.xlsx', 'db_monitor.xlsx']
# workbook = load_workbook(gestao_monitoramento)
# df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes')
# df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
# df_clientes.info()
# df_clientes.head()

In [69]:
def reporta_existencia(dataframe):
    # declara variavel para armazenar dados para relatório 
    global relatorio_global 

    # se receber df_cliente_certidao (nesse df, a col 'ARQUIVO' não tem a palavra .pdf)
    if not dataframe['NOME_REF'].str.contains('.pdf').any():

        df_false = dataframe[dataframe['EXISTE'] == False]
        
        if len(df_false) > 0: # se há linhas False

            dados = df_false.to_dict(orient='records')

            relatorio = {
                'cod': 'TESTE',
                'erro': 'Certidão não Encontrada',
                'mensagem': 'As seguintes certidões não foram encontradas nos arquivos',
                'dados': dados
            }
            relatorio_global.append(relatorio)
            print('relatorio adicionado')

    # se receber df_monitor
    else:
        print('tem .pdf')    
    

In [75]:
def processa_novos_arquivos(gestao_monitoramento):
    workbook = load_workbook(gestao_monitoramento)

    df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes', index_col=None)
    df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
    df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')
    df_monitor = df_monitor.dropna(how='all').reset_index(drop=True)



    # obtem lista unitária cliente 1:1 certidão
    df_cliente_certidao = certidoes_clientes_ativo(df_clientes, df_cert_config)

    # se retornou um dataframe
    if not df_cliente_certidao is None:
        # verifica se os arquivos existem na pasta_destino apontada
        df_cliente_certidao_resultado = verificar_existencia_arquivos(df_cliente_certidao)

        # reporta relatorio_global quais certidões não existem
        reporta_existencia(df_cliente_certidao_resultado)

        df_monitor_final = arquivos_novos_dbmonitor(df_monitor, df_cliente_certidao_resultado)

        try:
            atualizar_planilha(gestao_monitoramento, workbook, df_monitor)
        
        except PermissionError as e:
            if e.errno == 13:
                mensagem = f'Feche o arquivo "gestao_monitoramento.xlsx"\nErro: {e}'
                enviar_notificacao(f'Permissão Negada', mensagem, 10)
                return None
        except Exception as e:
                mensagem = f'Erro: {e}'
                enviar_notificacao(f'Erro ao atualizar planilha', mensagem, 10)
                return None
        return df_monitor_final
    
    else:
        print('não foi possível processar novas certidões')
        return None






df_monitor = processa_novos_arquivos(gestao_monitoramento)

ValueError: Worksheet named 'config_clientes' not found

In [71]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')


In [73]:
df_monitor.info()
df_monitor.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      15 non-null     object        
 1   CLIENTE_ID       15 non-null     object        
 2   PASTA_DESTINO    15 non-null     object        
 3   ARQUIVO          15 non-null     object        
 4   STATUS           15 non-null     object        
 5   DATA_INICIO      15 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  15 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  15 non-null     object        
 8   NADA_CONSTA      15 non-null     bool          
 9   CREATED_AT       15 non-null     datetime64[ns]
 10  UPDATED_AT       15 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 1.3+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,BIGBOX,clientes/bigbox/,2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-11-08,2024-12-07,2024110818421197249619,True,2024-11-16 02:51:45,2024-11-18 01:26:26
1,TJDFT-CNFR,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e R...,monitorando,2024-11-14,2024-12-14,2024.CTD.F2H1.0WMJ.OFPU.E6S2.0YBG,True,2024-11-16 02:51:45,2024-11-18 01:26:26
2,CGU-CNC,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (eP...,monitorando,2024-11-14,2024-12-14,qEcHlcYKFT1lcJeXQYcj,True,2024-11-16 02:51:45,2024-11-18 01:26:26
3,FN-CNDTF,BIGBOX,clientes/bigbox/,2025-04-14_(BIGBOX_FN-CNDTF)_CND Tributos Fede...,monitorando,2024-10-16,2025-04-14,71DD.4186.596B.A1FB,True,2024-11-16 02:51:45,2024-11-18 01:26:26
4,GDF-DIF,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_GDF-DIF)_Comprovante DIF GD...,monitorando,2024-11-14,2024-12-14,-não existe-,True,2024-11-16 02:51:45,2024-11-18 01:26:26
5,TST-CNDT,BIGBOX,clientes/bigbox/,2025-05-13_(BIGBOX_TST-CNDT)_CND Trabalhista.pdf,monitorando,2024-11-14,2025-05-13,78855115/2024,True,2024-11-16 02:51:45,2024-11-18 01:26:26
6,CNJ-CNIADM,BIGBOX,clientes/bigbox/,2024-12-14_(BIGBOX_CNJ-CNIADM)_CN Impropidade ...,monitorando,2024-11-14,2024-12-14,6735.7332.D1E2.8146,True,2024-11-16 02:51:45,2024-11-18 01:26:26
7,CEF-CRFGTS,DROSARIO,clientes/drosario/,2024-11-29_(DROSARIO_CEF-CRFGTS)_CR FGTS.pdf,monitorando,2024-10-31,2024-11-29,2024103110152970642061,True,2024-11-16 02:51:45,2024-11-18 01:26:26
8,TJDFT-CNFR,DROSARIO,clientes/drosario/,2024-12-14_(DROSARIO_TJDFT-CNFR)_CN Falencia e...,monitorando,2024-11-14,2024-12-14,2024.CTD.ZTPK.TE86.3MSB.AT0F.SMCG,True,2024-11-16 02:51:45,2024-11-18 01:26:26
9,CGU-CNC,DROSARIO,clientes/drosario/,2024-12-14_(DROSARIO_CGU-CNC)_CN Correcional (...,monitorando,2024-11-14,2024-12-14,Frp6kCRkte8Vkevk9Lar,True,2024-11-16 02:51:45,2024-11-18 01:26:26


# a



# tentativa


In [243]:
relatorio_global = []

In [247]:
print(len(relatorio_global))
print(relatorio_global)

0
[]


In [215]:
def reporta_existencia(dataframe):
    # declara variavel para armazenar dados para relatório 
    global relatorio_global 

    # se receber df_cliente_certidao (nesse df, a col 'ARQUIVO' não tem a palavra .pdf)
    if not dataframe['NOME_REF'].str.contains('.pdf').any():

        df_false = dataframe[dataframe['EXISTE'] == False]
        
        if len(df_false) > 0: # se há linhas False

            dados = df_false.to_dict(orient='records')

            relatorio = {
                'cod': 'TESTE',
                'erro': 'Certidão não Encontrada',
                'mensagem': 'As seguintes certidões não foram encontradas nos arquivos',
                'dados': dados
            }
            relatorio_global.append(relatorio)
            print('relatorio adicionado')

    # se receber df_monitor
    else:
        print('tem .pdf')    

In [ ]:
def atualizar_planilha(gestao_monitoramento, workbook, df_monitor):
    # Carrega o arquivo Excel
    workbook = load_workbook(gestao_monitoramento)
    
    # Seleciona a aba 'db_monitor'
    sheet = workbook['db_monitor']

    # Localiza a tabela 'fDB_monitor'
    table = None
    for tbl in sheet._tables.values():
        if tbl.name == 'fDB_monitor':
            table = tbl
            break

    if table is None:
        print("Tabela 'fDB_monitor' não encontrada.")
        return

    # Limpa todas as linhas da tabela (exceto o cabeçalho)
    start_row = table.ref.split(':')[0][1:]
    end_row = table.ref.split(':')[1][1:]
    start_col = table.ref.split(':')[0][0]
    end_col = table.ref.split(':')[1][0]

    for row in sheet.iter_rows(min_row=int(start_row) + 1, max_row=int(end_row), min_col=ord(start_col) - 64, max_col=ord(end_col) - 64):
        for cell in row:
            cell.value = None

    # Preenche os dados do DataFrame a partir da linha da tabela
    for index, row in df_monitor.iterrows():
        for col_num, value in enumerate(row, start=1):  # start=1 para a primeira coluna (A)
            sheet.cell(row=int(start_row) + 1 + index, column=col_num, value=value)  # Inicia na linha da tabela, logo abaixo do cabeçalho

    # Atualiza o intervalo da tabela para incluir todas as novas linhas
    table.ref = f"{start_col}{start_row}:{end_col}{int(start_row) + len(df_monitor)}"

    # Salva as alterações no arquivo Excel
    workbook.save(gestao_monitoramento)
    print("Arquivo Excel atualizado com os dados do DataFrame!")

# Exemplo de uso
gestao_monitoramento = 'gestao_monitoramento.xlsx'
workbook = load_workbook(gestao_monitoramento)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

# atualizar_planilha(gestao_monitoramento, workbook, df_monitor_final)

In [262]:
# carrega dataframes
df_cert_config = pd.read_excel(config_bot, sheet_name='config_certidoes', index_col=None)
df_clientes = pd.read_excel(config_bot, sheet_name='config_clientes', index_col=None)
df_clientes['CERTIDOES_CADASTRADAS'] = df_clientes['CERTIDOES_CADASTRADAS'].astype(str)

# explode em df_clientes a coluna 'CERTIDOES_CADASTRADAS' para nova coluna'CERTIDAO_ID'
df_clientes_expandido = df_clientes.assign(
    CERTIDAO_ID=df_clientes['CERTIDOES_CADASTRADAS'].str.split(', ')
    ).explode('CERTIDAO_ID')

# merge dos dados df_clientes_expandido e df_cert_config, selecionando as colunas elencadas
df_clientes_certidoes = pd.merge(df_clientes_expandido,
                                    df_cert_config,
                                    on='CERTIDAO_ID',
                                    how='left'
                                    )[['CERTIDAO_ID', 'CLIENTE_ID', 'ATIVO', 'PASTA_DESTINO', 'NOME_ARQUIVO']]

# verifica se existem valores NaN nas colunas 'CERTIDAO_ID', 'NOME_ARQUIVO' e dropa essas linhas
if df_clientes_certidoes[['CERTIDAO_ID', 'NOME_ARQUIVO']].isna().any().any():
    print('Existem valores NaN nas colunas CERTIDAO_ID ou NOME_ARQUIVO.')

    df_clientes_certidoes = df_clientes_certidoes.dropna(subset=['CERTIDAO_ID', 'NOME_ARQUIVO'])

    relatorio = {
        'cod': 'CC11-A',
        'erro': 'Dados faltantes em "config_bot.xlsx"',
        'mensagem': 'Verifique a relação/existência de dados entre as colunas "CERTIDAO_ID" e "CERTIDOES_CADASTRADAS"',
    }
df_clientes_certidoes

,CERTIDAO_ID,CLIENTE_ID,ATIVO,PASTA_DESTINO,NOME_ARQUIVO
0,CEF-CRFGTS,AUTOLTDA,True,clientes/autodatas//,CR FGTS
1,TJDFT-CNFR,AUTOLTDA,True,clientes/autodatas//,CN Falencia e Recuperacao
2,CGU-CNC,AUTOLTDA,True,clientes/autodatas//,"CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM)"
3,FN-CNDTF,AUTOLTDA,True,clientes/autodatas//,CND Tributos Federais
4,GDF-CND,AUTOLTDA,True,clientes/autodatas//,CND GDF
5,GDF-DIF,AUTOLTDA,True,clientes/autodatas//,Comprovante DIF GDF
6,TST-CNDT,AUTOLTDA,True,clientes/autodatas//,CND Trabalhista
7,CNJ-CNIADM,AUTOLTDA,True,clientes/autodatas//,CN Impropidade Administrativa e Inelegibilidade
8,TESTE-TESTE,AUTOLTDA,True,clientes/autodatas//,TesteInexistente
9,CEF-CRFGTS,BOTMEI,True,clientes/botmeio/,CR FGTS


In [311]:
def certidoes_clientes_ativo(arquivo_xlsx): # ATUALIZZADA 2
    '''
    Cria DF com lista cliente 1:1 certidão. Remove valores nulos e reporta erros encontrados.
    Retorna None em caso de erro ou tamanho DF gerado == 0.
    '''

    # declara variável para armazenadas dados para relatório
    global relatorio_global
    relatorio = {}

    try:
        # carrega dataframes
        df_cert_config = pd.read_excel(arquivo_xlsx, sheet_name='config_certidoes', index_col=None)
        df_clientes = pd.read_excel(arquivo_xlsx, sheet_name='config_clientes', index_col=None)
        df_clientes['CERTIDOES_CADASTRADAS'] = df_clientes['CERTIDOES_CADASTRADAS'].astype(str)
        
        # explode em df_clientes a coluna 'CERTIDOES_CADASTRADAS' para nova coluna'CERTIDAO_ID'
        df_clientes_expandido = df_clientes.assign(
            CERTIDAO_ID=df_clientes['CERTIDOES_CADASTRADAS'].str.split(', ')
            ).explode('CERTIDAO_ID')

        # merge dos dados df_clientes_expandido e df_cert_config, selecionando as colunas elencadas
        df_clientes_certidoes = pd.merge(df_clientes_expandido,
                                         df_cert_config,
                                         on='CERTIDAO_ID',
                                         how='left'
                                         )[['CERTIDAO_ID', 'CLIENTE_ID', 'ATIVO', 'PASTA_DESTINO', 'NOME_ARQUIVO']]
        
        # verifica se existem valores NaN nas colunas 'CERTIDAO_ID', 'NOME_ARQUIVO' e dropa essas linhas
        if df_clientes_certidoes[['CERTIDAO_ID', 'NOME_ARQUIVO']].isna().any().any():
            print('Existem valores NaN nas colunas CERTIDAO_ID ou NOME_ARQUIVO.')

            df_clientes_certidoes = df_clientes_certidoes.dropna(subset=['CERTIDAO_ID', 'NOME_ARQUIVO'])

            relatorio = {
                'cod': 'CC11-A',
                'erro': 'Dados faltantes em "config_bot.xlsx"',
                'mensagem': 'Verifique a relação/existência de dados entre as colunas "CERTIDAO_ID" e "CERTIDOES_CADASTRADAS"',
            }

        # se tamanho DF == 0 reporta e encerra o processo com None
        if len(df_clientes_certidoes) == 0:
            print('Não foi possível gerar relação cliente x certidao.')

            relatorio = {
                'cod': 'CC11-B',
                'erro': 'Erro ao gerar relação cliente x certidao',
                'mensagem': 'Preencha corretamente as planilhas "cert_config" e "cert_clientes" do arquivo "config_bot.xlsx".',
            }
            return None
        
        else:
            # renomeia a coluna para o nome ficar igual ao do arquivo 'db_monitor'
            df_clientes_certidoes = df_clientes_certidoes.rename(columns={'NOME_ARQUIVO': 'ARQUIVO'})
            # filtrando apenas onde o 'ATIVO' é True
            df_clientes_certidoes = df_clientes_certidoes.loc[df_clientes_certidoes['ATIVO'] == True]
        
            print('df_clientes_certidoes criado com sucesso')
            return df_clientes_certidoes
    
    except Exception as e:
        relatorio = {
            'cod': 'CC11-E',
            'erro': 'Erro desconhecido ao gerar relação cliente x certidao',
            'mensagem': f'Erro: {e}'
        }
        return None
    
    finally:
        if len(relatorio) > 0:
                relatorio_global.append(relatorio)

# ATUALIZADA 2
config_bot = 'config_bot.xlsx'

df_clientes_certidoes = certidoes_clientes_ativo(config_bot)

df_clientes_certidoes criado com sucesso


In [324]:
# ATUALIZADA
# declara variavel para armazenar dados para relatório 
global relatorio_global 
relatorio = []

try:
    resultados = [] # variavel para armazenar resultados da consulta
    # agrupa DF pela coluna 'PASTA_DESTINO'
    pastas_grupo = df_clientes_certidoes.groupby('PASTA_DESTINO')

    # iterando DF por 'PASTA_DESTINO'
    for pasta_destino, grupo in pastas_grupo:

        # condição caso o diretório indicado na 'PASTA_DESTINO' não exista
        if not os.path.isdir(pasta_destino):
            print(f'não existe {pasta_destino} do cliente {row["CLIENTE_ID"]}')

            # anexa msg para relatório global
            relatorio = {
                 'cod': 'VERIARQ-A',
                 'erro': 'Diretório não encontrado',
                 'mensagem': f'Cliente: {row["CLIENTE_ID"]}. Diretório -- {row["PASTA_DESTINO"]} -- inexistente'
                 }
            relatorio_global.append(relatorio)
            
            # itera nas linhas referentes a 'PASTA_DESTINO' agrupada e adiciona ao resultados
            for _, row in grupo.iterrows():
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'NOME_REF': row['ARQUIVO'],
                        'ARQUIVO_ENCONTRADO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
        else:
            print(f'existe {pasta_destino}')
        print()

        
#             for _, row in grupo.iterrows():
#                 resultados.append({
#                     'CERTIDAO_ID': row['CERTIDAO_ID'],
#                     'CLIENTE_ID': row['CLIENTE_ID'],
#                     'EXISTE': False,
#                     'NOME_REF': row['ARQUIVO'],
#                     'ARQUIVO_ENCONTRADO': None,
#                     'PASTA_DESTINO': pasta_destino
#                 })
            
#             relatorio = {
#                 'cod': 'FATAL02',
#                 'erro': 'Diretório não encontrado',
#                 'mensagem': f'Não foi possível encontrar o diretório: {pasta_destino}',
#                 'dados': None
#             }
#             continue

#         # lista todos os arquivos da pasta_destino
#         arquivos_na_pasta = os.listdir(pasta_destino)

#         for _, row in grupo.iterrows():
#             nome_arquivo = row['ARQUIVO']
#             arquivos_encontrados = []

#             for arquivo in arquivos_na_pasta:
#                 if nome_arquivo in arquivo and arquivo.endswith('.pdf'):
#                     arquivo_pdf = os.path.join(arquivo)
#                     arquivos_encontrados.append(arquivo_pdf)

#             # Adicionar uma entrada para cada arquivo encontrado
#             if arquivos_encontrados:
#                 for arquivo_pdf in arquivos_encontrados:
#                     resultados.append({
#                         'CERTIDAO_ID': row['CERTIDAO_ID'],
#                         'CLIENTE_ID': row['CLIENTE_ID'],
#                         'EXISTE': True,
#                         'NOME_REF': row['ARQUIVO'],
#                         'ARQUIVO_ENCONTRADO': arquivo_pdf,
#                         'PASTA_DESTINO': pasta_destino
#                     })
#             else:
#                 # Se nenhum arquivo foi encontrado, adicionar como não encontrado
#                 resultados.append({
#                     'CERTIDAO_ID': row['CERTIDAO_ID'],
#                     'CLIENTE_ID': row['CLIENTE_ID'],
#                     'EXISTE': False,
#                     'NOME_REF': row['ARQUIVO'],
#                     'ARQUIVO_ENCONTRADO': None,
#                     'PASTA_DESTINO': pasta_destino
#                 })
#     #cria df com os resultados encontrados
#     df_cliente_certidoe_resultado = pd.DataFrame(resultados)
#     return df_cliente_certidoe_resultado
except Exception as e:
    print(f'Erro ao verificar a existência dos arquivos {e}')

não existe clientes/autodatas// do cliente AUTOLTDA

existe clientes/bigbox/

existe clientes/botmei/

existe clientes/drosario/

existe clientes/testeltda/



In [325]:
print(relatorio)
print(resultados)
print(relatorio_global)

{'cod': 'VERIARQ-A', 'erro': 'Diretório não encontrado', 'mensagem': 'Cliente: AUTOLTDA. Diretório -- clientes/autodatas// -- inexistente'}
[{'CERTIDAO_ID': 'CEF-CRFGTS', 'CLIENTE_ID': 'AUTOLTDA', 'EXISTE': False, 'NOME_REF': 'CR FGTS', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/autodatas//'}, {'CERTIDAO_ID': 'TJDFT-CNFR', 'CLIENTE_ID': 'AUTOLTDA', 'EXISTE': False, 'NOME_REF': 'CN Falencia e Recuperacao', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/autodatas//'}, {'CERTIDAO_ID': 'CGU-CNC', 'CLIENTE_ID': 'AUTOLTDA', 'EXISTE': False, 'NOME_REF': 'CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM)', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/autodatas//'}, {'CERTIDAO_ID': 'FN-CNDTF', 'CLIENTE_ID': 'AUTOLTDA', 'EXISTE': False, 'NOME_REF': 'CND Tributos Federais', 'ARQUIVO_ENCONTRADO': None, 'PASTA_DESTINO': 'clientes/autodatas//'}, {'CERTIDAO_ID': 'GDF-CND', 'CLIENTE_ID': 'AUTOLTDA', 'EXISTE': False, 'NOME_REF': 'CND GDF', 'ARQUIVO_ENCONTRADO': None, 'PAST

In [285]:
# ATUALIZADA
def verificar_existencia_arquivos(dataframe):
    '''
    Recebe DF e verifica se o arquivo indicado existe na pasta destino.
    Retorna DF indicando col 'EXISTE' True ou False
    '''
    # declara variavel para armazenar dados para relatório 
    global relatorio_global 
    relatorios = []

    try:
        
        resultados = []

        # agrupa as ocorrencias pela coluna 'PASTA_DESTINO'
        pastas_grupo = dataframe.groupby('PASTA_DESTINO')

        for pasta_destino, grupo in pastas_grupo:
            # verificar se o diretório existe
            if not os.path.isdir(pasta_destino):
                for _, row in grupo.iterrows():
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'NOME_REF': row['ARQUIVO'],
                        'ARQUIVO_ENCONTRADO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
                
                relatorio = {
                    'cod': 'VERIARQ-A',
                    'erro': 'Diretório não encontrado',
                    'mensagem': f'Ao verificar existência do arquivo, não foi possível encontrar o diretório: {pasta_destino}'
                }
                relatorios.append(relatorio)
                continue

            # lista todos os arquivos da pasta_destino
            arquivos_na_pasta = os.listdir(pasta_destino)

            for _, row in grupo.iterrows():
                nome_arquivo = row['ARQUIVO']
                arquivos_encontrados = []

                for arquivo in arquivos_na_pasta:
                    if nome_arquivo in arquivo and arquivo.endswith('.pdf'):
                        arquivo_pdf = os.path.join(arquivo)
                        arquivos_encontrados.append(arquivo_pdf)

                # Adicionar uma entrada para cada arquivo encontrado
                if arquivos_encontrados:
                    for arquivo_pdf in arquivos_encontrados:
                        resultados.append({
                            'CERTIDAO_ID': row['CERTIDAO_ID'],
                            'CLIENTE_ID': row['CLIENTE_ID'],
                            'EXISTE': True,
                            'NOME_REF': row['ARQUIVO'],
                            'ARQUIVO_ENCONTRADO': arquivo_pdf,
                            'PASTA_DESTINO': pasta_destino
                        })
                else:
                    # Se nenhum arquivo foi encontrado, adicionar como não encontrado
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'NOME_REF': row['ARQUIVO'],
                        'ARQUIVO_ENCONTRADO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
        #cria df com os resultados encontrados
        df_cliente_certidoe_resultado = pd.DataFrame(resultados)
        return df_cliente_certidoe_resultado
    except Exception as e:
        print(f'Erro ao verificar a existência dos arquivos {e}')
    finally:
        print(relatorios)

df_cliente_certidao_resultado = verificar_existencia_arquivos(df_clientes_certidoes)

Erro ao verificar a existência dos arquivos 'ARQUIVO'
[]


In [ ]:
def processa_novos_arquivos():

    df_clientes = pd.read_excel('config_bot.xlsx', sheet_name='config_clientes', index_col=None)
    df_cert_config = pd.read_excel('config_bot.xlsx', sheet_name='config_certidoes', index_col=None)
    df_monitor = pd.read_excel('db_monitor.xlsx', sheet_name='db_monitor')
    df_monitor = df_monitor.dropna(how='all').reset_index(drop=True)



    # obtem lista unitária cliente 1:1 certidão
    df_cliente_certidao = certidoes_clientes_ativo(df_clientes, df_cert_config)

    # se retornou um dataframe
    if not df_cliente_certidao is None:
        # verifica se os arquivos existem na pasta_destino apontada
        df_cliente_certidao_resultado = verificar_existencia_arquivos(df_cliente_certidao)

        # reporta relatorio_global quais certidões não existem
        reporta_existencia(df_cliente_certidao_resultado)

        df_monitor_final = arquivos_novos_dbmonitor(df_monitor, df_cliente_certidao_resultado)

        try:
            atualizar_planilha(gestao_monitoramento, workbook, df_monitor)
        
        except PermissionError as e:
            if e.errno == 13:
                mensagem = f'Feche o arquivo "gestao_monitoramento.xlsx"\nErro: {e}'
                enviar_notificacao(f'Permissão Negada', mensagem, 10)
                return None
        except Exception as e:
                mensagem = f'Erro: {e}'
                enviar_notificacao(f'Erro ao atualizar planilha', mensagem, 10)
                return None
        return df_monitor_final
    
    else:
        print('não foi possível processar novas certidões')
        return None






df_monitor = processa_novos_arquivos(gestao_monitoramento)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CERTIDAO_ID    46 non-null     object 
 1   CLIENTE_ID     46 non-null     object 
 2   ATIVO          0 non-null      float64
 3   PASTA_DESTINO  46 non-null     object 
 4   ARQUIVO        45 non-null     object 
dtypes: float64(1), object(4)
memory usage: 1.9+ KB
df_cliente_certidao criado com sucesso


KeyError: 'NOME_REF'